# Chapter 03. 머신러닝 분류 문제 연습하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 실습 가이드
1. 데이터를 다운로드하여 Colab에 불러옵니다.
> 다운로드한 데이터의 위치는 **Colab Notebooks/data/** 로 통일합니다.

2. 필요한 라이브러리는 모두 코드로 작성되어 있습니다.
3. 코드는 위에서부터 아래로 순서대로 실행합니다.
4. 전체 문제 구성은 좌측 첫 번째 아이콘을 통해 확인할 수 있습니다.

### <b> 데이터의 속성</b>
**시작 전에** 원본 데이콘 페이지에서 상세한 데이터에 대한 설명을 확인하세요 !

Reference : https://dacon.io/competitions/official/9565/data


### <b>학습목표</b>
- 1) Binary Classification 모델의 이해
- 2) 학습했던 다양한 머신러닝 분류모델의 이해
- 3) 분류를 위한 데이터 전처리 방법에 대한 이해
- 4) feature engineering에 대한 이해
- 5) 평가결과를 바탕으로 모델을 개선하는 방법 습득

- 출제자: 김용담 강사

## Step 1. 예측할 데이터 불러오기

### 문제 01. 데이터 불러오기

In [ ]:
# 분석에 필요한 라이브러리를 불러옵니다
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/data/9565_hospital_data'
train = pd.read_csv(base_path + "/train.csv")
test = pd.read_csv(base_path + "/test.csv")

### 문제 02. 데이터 미리보기

In [ ]:
# 데이터 크기 확인
print(train.shape, test.shape)

# 데이터 일부 확인
display(train.head())
display(test.head())

(301, 58) (127, 58)


,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.0,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,31641798.0,225169678.0,1.012700e+09,9.976719e+08,4.700557e+08,1.502781e+07,2.514586e+09,2.360684e+09,1.434496e+08,0.0,6.828260e+08,2.013237e+08,0.000000e+00,4.815023e+08,3.510000e+08,2.844460e+09,1.496394e+09,4.297848e+09,0.0,4.057422e+09,2.063787e+09,16194675.0,2.998335e+07,1.568305e+07,18808074.0,207829685.0,8.301695e+08,8.165705e+08,5.237026e+08,1.359897e+07,2.548115e+09,2.386263e+09,1.458986e+08,0.0,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,515483669.0,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,0.0,41864754.0,2.724421e+08,2.536822e+08,8.095950e+06,1.875997e+07,1.204810e+08,1.204810e+08,0.000000e+00,0.0,9.241434e+07,9.241434e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.005088e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.0,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,173769780.0,902830288.0,1.304154e+10,1.153475e+10,0.000000e+00,1.506793e+09,4.317936e+10,3.832078e+10,3.945208e+09,0.0,4.236380e+10,2.799188e+10,1.988520e+10,1.437192e+10,7.253040e+09,1.385710e+10,8.643659e+09,6.685834e+10,0.0,6.492419e+10,2.971135e+10,476807804.0,1.353672e+09,1.277422e+09,218891720.0,838387466.0,1.112572e+10,9.890540e+09,0.000000e+00,1.235181e+09,3.958356e+10,3.485576e+10,3.915906e+09,0.0,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.0,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,0.0,80749696.0,6.317084e+09,5.873265e+09,4.099320e+09,4.438186e+08,4.366733e+10,4.330613e+10,2.223400e+08,0.0,4.989811e+10,2.890919e+10,1.793038e+10,2.098892e+10,1.350000e+10,8.631164e+07,9.025550e+09,4.808280e+10,0.0,4.712580e+10,2.346004e+10,597748128.0,1.522108e+09,1.349851e+09,0.0,32642585.0,4.906776e+09,4.464017e+09,3.365227e+09,4.427591e+08,4.653138e+10,4.562945e+10,7.893407e+08,0.0,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same


,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,2,NaN,incheon,139,19981125.0,300.0,general_hospital,6.682486e+10,0.000000e+00,6.565709e+10,3.223695e+10,4.314093e+09,4.901517e+09,1.775872e+09,76680000.0,5.036692e+08,2.591385e+10,2.438741e+10,7.242580e+08,1.526434e+09,4.807282e+10,4.629931e+10,7.463122e+08,0.0,6.769090e+10,5.426088e+10,3.805927e+10,1.343002e+10,6.319097e+09,6.295769e+09,8.944587e+09,6.360375e+10,0.000000e+00,6.261126e+10,3.016946e+10,4.076448e+09,4.332357e+09,1.578626e+09,218399804.0,5.181821e+08,2.498992e+10,2.367928e+10,1.237754e+09,1.310637e+09,4.650191e+10,4.356296e+10,796386240.0,0.0,5.540643e+10,5.068443e+10,3.714334e+10,4.720000e+09,4.690000e+09,1.608540e+10,8.944587e+09,693,693,same
1,5,NaN,jeju,149,20160309.0,44.0,hospital,3.495758e+10,0.000000e+00,3.259270e+10,1.672254e+10,6.871024e+07,1.981033e+09,1.936455e+09,0.0,4.525557e+08,5.861727e+09,5.372198e+09,6.610290e+08,4.895291e+08,7.511010e+10,6.678648e+10,4.500000e+06,0.0,6.626737e+10,4.210945e+10,2.330000e+10,2.415791e+10,1.720000e+10,1.470447e+10,1.046277e+10,3.754253e+10,0.000000e+00,3.868523e+10,1.897732e+10,1.918022e+08,3.694365e+09,2.016114e+09,0.0,-4.645265e+09,5.503720e+09,4.943368e+09,1.544307e+08,5.603519e+08,6.559379e+10,6.545302e+10,14500000.0,0.0,6.730838e+10,4.209828e+10,2.420000e+10,2.521009e+10,1.830000e+10,3.789135e+09,0.000000e+00,379,371,same
2,6,NaN,jeonnam,103,19890427.0,276.0,general_hospital,2.326031e+10,2.542571e+09,2.308749e+10,1.168734e+10,1.601943e+09,1.380941e+09,8.101204e+08,0.0,-2.148749e+09,1.265313e+10,1.133922e+10,7.337107e+09,1.313914e+09,2.945467e+10,2.830632e+10,1.388070e+08,0.0,0.000000e+00,2.830632e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,2.468035e+10,1.500100e+09,2.373950e+10,1.191146e+10,2.361216e+08,1.328858e+09,6.259234e+08,15210270.0,-1.667198e+09,1.354202e+10,1.263851e+10,7.003703e+09,9.035036e+08,2.938810e+10,2.777589e+10,428850980.0,0.0,0.000000e+00,2.777589e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN,NaN
3,8,NaN,busan,71,20100226.0,363.0,general_hospital,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.850652e+09,2.060989e+09,0.000000e+00,0.0,2.789663e+09,8.644533e+09,7.891338e+09,0.000000e+00,7.531944e+08,6.730264e+10,6.415297e+10,0.000000e+00,0.0,2.891880e+10,2.604193e+10,1.875187e+10,2.876879e+09,0.000000e+00,4.702837e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.975530e+09,2.112736e+09,0.000000e+00,0.0,2.862794e+09,1.204897e+10,1.126481e+10,0.000000e+00,7.841551e+08,3.920904e+10,3.601937e+10,0.0,0.0,1.211517e+10,9.556237e+09,4.251867e+09,2.558931e+09,0.000000e+00,3.914284e+10,0.000000e+00,760,760,same
4,10,NaN,jeonbuk,26,20040604.0,213.0,general_hospital,5.037025e+10,0.000000e+00,4.855803e+10,2.296346e+10,1.534452e+09,0.000000e+00,2.573804e+09,0.0,3.346675e+09,7.849772e+09,7.138327e+09,1.104000e+08,7.114450e+08,6.404854e+10,6.024301e+10,3.582019e+09,0.0,4.921637e+10,2.503353e+10,1.720219e+10,2.418283e+10,9.976421e+09,2.268194e+10,1.492691e+10,4.348996e+10,0.000000e+00,4.067810e+10,1.871616e+10,1.802199e+09,0.000000e+00,3.925279e+09,0.0,4.614056e+09,8.847725e+09,7.868784e+09,1.764000e+08,9.789409e+08,6.085689e+10,6.014111e+10,546748880.0,0.0,4.395973e+10,7.535567e+09,3.298427e+09,3.642417e+10,2.134712e+10,2.574488e+10,1.507269e+10,437,385,same


### 문제 03. 데이터 확인하기

In [ ]:
# train 데이터에서 결측치가 있는 column 확인하기
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    object 
 2   sido                301 non-null    object 
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            296 non-null    float64
 6   instkind            300 non-null    object 
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

In [ ]:
# test도 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             127 non-null    int64  
 1   OC                  0 non-null      float64
 2   sido                127 non-null    object 
 3   sgg                 127 non-null    int64  
 4   openDate            126 non-null    float64
 5   bedCount            119 non-null    float64
 6   instkind            125 non-null    object 
 7   revenue1            125 non-null    float64
 8   salescost1          125 non-null    float64
 9   sga1                125 non-null    float64
 10  salary1             125 non-null    float64
 11  noi1                125 non-null    float64
 12  noe1                125 non-null    float64
 13  interest1           125 non-null    float64
 14  ctax1               125 non-null    float64
 15  profit1             125 non-null    float64
 16  liquidAs

## Step 2. Data Preprocessing

- 데이터에 있는 결측치를 처리합니다.

- 결측치를 처리하기 위해 데이터에 잘못 기록된 점들을 확인합니다.

### 문제 04. train data에서 결측치가 있는 데이터 뽑기

In [ ]:
# train 데이터에서 결측치가 있는 column들을 뽑습니다.
train_missing_cols = train.columns[train.isnull().any()]
train_missing_cols

Index(['bedCount', 'instkind', 'revenue1', 'salescost1', 'sga1', 'salary1',
       'noi1', 'noe1', 'interest1', 'ctax1', 'profit1', 'liquidAsset1',
       'quickAsset1', 'receivableS1', 'inventoryAsset1', 'nonCAsset1',
       'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1',
       'liquidLiabilities1', 'shortLoan1', 'NCLiabilities1', 'longLoan1',
       'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2',
       'noi2', 'noe2', 'interest2', 'ctax2', 'profit2', 'liquidAsset2',
       'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2',
       'tanAsset2', 'OnonCAsset2', 'receivableL2', 'debt2',
       'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2',
       'netAsset2', 'surplus2', 'employee1', 'employee2', 'ownerChange'],
      dtype='object')

In [ ]:
# 데이터가 거의 비어있는 row가 있는지 확인합니다.
missing_rows = train[train.isnull().any(axis=1)]
missing_rows

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
14,26,open,ulsan,127,20160531,237.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,951.0,901.0,same
32,55,open,incheon,141,19930708,232.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,323.0,323.0,same
35,58,close,gyeonggi,168,20130206,132.0,nursing_hospital,2.681766e+09,1.802009e+08,2.014580e+09,9.366451e+08,59924069.0,4.287212e+08,284069476.0,0.0,1.181879e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.093132e+09,1.428541e+08,1.751390e+09,9.164822e+08,21947086.0,1.472983e+08,113100675.0,0.0,7.353718e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,51.0,NaN,change
48,74,open,seoul,1,19940425,28.0,traditional_hospital,8.872427e+09,9.641249e+08,7.638920e+09,4.488958e+09,19153521.0,2.160889e+08,204479315.0,72446781.0,0.000000e+00,1.302115e+09,1.186743e+09,2.671059e+06,97648359.0,8.347311e+09,8.178953e+09,2200500.0,0.0,6.749132e+09,5.759132e+09,4.000000e+09,9.900000e+08,9.900000e+08,2.900294e+09,0.000000e+00,8.632333e+09,8.994174e+08,7.616309e+09,4.283771e+09,132492468.0,2.188810e+08,199357358.0,30218421.0,0.000000e+00,1.002794e+09,8.915144e+08,0.000000e+00,80868500.0,8.581435e+09,8.368729e+09,25520500.0,0.0,5.958278e+09,5.956278e+09,4.992758e+09,2.000000e+06,0.000000e+00,3.625952e+09,0.000000e+00,NaN,NaN,NaN
55,87,open,gyeonggi,120,20130314,327.0,nursing_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,203.0,same
60,94,open,gangwon,128,20031201,244.0,nursing_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,96,open,busan,64,20170602,128.0,nursing_hospital,5.236406e+08,2.306773e+07,8.355775e+08,5.644636e+08,7534205.0,5.263060e+07,52630597.0,0.0,-3.801011e+08,6.073662e+08,5.953914e+08,1.015000e+06,11974763.0,3.629704e+09,3.629704e+09,0.0,0.0,3.513950e+09,3.342069e+09,2.918153e+09,1.718817e+08,1.718817e+08,7.231197e+08,7.231197e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,38.0,NaN,NaN
71,107,close,jeonbuk,26,19990805,NaN,traditional_hospital,4.131858e+09,1.436118e+08,3.529097e+09,2.198043e+09,56630198.0,2.281945e+08,113598011.0,22221340.0,2.653634e+08,1.970952e+09,1.963787e+09,0.000000e+00,7165400.0,3.553691e+09,2.7499

In [ ]:
# missing_rows중에 2개 이상의 column이 비어있는 데이터는 삭제합니다.
missing_twos = missing_rows[missing_rows.isnull().sum(axis=1) > 2]
#missing_twos.index
train.drop(missing_twos.index, inplace=True)
train

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,31641798.0,2.251697e+08,1.012700e+09,9.976719e+08,4.700557e+08,1.502781e+07,2.514586e+09,2.360684e+09,1.434496e+08,0.0,6.828260e+08,2.013237e+08,0.000000e+00,4.815023e+08,3.510000e+08,2.844460e+09,1.496394e+09,4.297848e+09,0.000000e+00,4.057422e+09,2.063787e+09,16194675.0,2.998335e+07,1.568305e+07,18808074.0,2.078297e+08,8.301695e+08,8.165705e+08,5.237026e+08,1.359897e+07,2.548115e+09,2.386263e+09,1.458986e+08,0.0,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,0.0,4.186475e+07,2.724421e+08,2.536822e+08,8.095950e+06,1.875997e+07,1.204810e+08,1.204810e+08,0.000000e+00,0.0,9.241434e+07,9.241434e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.005088e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,173769780.0,9.028303e+08,1.304154e+10,1.153475e+10,0.000000e+00,1.506793e+09,4.317936e+10,3.832078e+10,3.945208e+09,0.0,4.236380e+10,2.799188e+10,1.988520e+10,1.437192e+10,7.253040e+09,1.385710e+10,8.643659e+09,6.685834e+10,0.000000e+00,6.492419e+10,2.971135e+10,476807804.0,1.353672e+09,1.277422e+09,218891720.0,8.383875e+08,1.112572e+10,9.890540e+09,0.000000e+00,1.235181e+09,3.958356e+10,3.485576e+10,3.915906e+09,0.0,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,0.0,8.074970e+07,6.317084e+09,5.873265e+09,4.099320e+09,4.438186e+08,4.366733e+10,4.330613e+10,2.223400e+08,0.0,4.989811e+10,2.890919e+10,1.793038e+10,2.098892e+10,1.350000e+10,8.631164e+07,9.025550e+09,4.808280e+10,0.000000e+00,4.712580e+10,2.346004e+10,597748128.0,1.522108e+09,1.349851e+09,0.0,3.264258e+07,4.906776e+09,4.464017e+09,3.365227e+09,4.427591e+08,4.653138e+10,4.562945e+10,7.893407e+08,0.0,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same
5,11,open,gyeongnam,155,20020501,243.0,general_hospital,3.358054e+10,9.222997e+09,2.372791e+10,1.665533e+10,65414325.0,7.412694e+08,3.364622e+08,0.0,-4.622738e+07,5.635105e+09,5.481680e+09,6.634820e+08,1.534246e+08,1.864970e+10,1.849938e+10,1.364582e+08,0.0,0.000000e+00,1.849938e+10,1.718053e+10,0.000000e+00,0.000000e+00,0.000000e+00,3.664918e+09,3.433445e+10,9.839139e+09,2.409622e+10,1.638792e+10,125681154.0,4.500830e+08,4.155104e+08,0.0,7.468595e+07,4.869419e+09,4.725857e+09,1.976163e+08,1.435618e+08,2.212715e+10,2.186113e+10,1.910460e+08,0.0,0.000000e+00,2.186113e+10,1.708714e+10,2.000000e+07,0.000000e+00,0.000000e+00,5.879004e+09,397.0,390.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# 여전히 결측치가 있는 데이터를 찾아봅시다.
missings = train[train.isnull().any(axis=1)]
missings

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
35,58,close,gyeonggi,168,20130206,132.0,nursing_hospital,2.681766e+09,1.802009e+08,2.014580e+09,9.366451e+08,59924069.0,428721218.0,284069476.0,0.0,1.181879e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.093132e+09,142854148.0,1.751390e+09,9.164822e+08,21947086.0,1.472983e+08,113100675.0,0.0,7.353718e+07,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,51.0,NaN,change
62,96,open,busan,64,20170602,128.0,nursing_hospital,5.236406e+08,2.306773e+07,8.355775e+08,5.644636e+08,7534205.0,52630597.0,52630597.0,0.0,-3.801011e+08,6.073662e+08,5.953914e+08,1015000.0,11974763.0,3.629704e+09,3.629704e+09,0.0,0.0,3.513950e+09,3.342069e+09,2.918153e+09,1.718817e+08,1.718817e+08,7.231197e+08,7.231197e+08,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,38.0,NaN,NaN
71,107,close,jeonbuk,26,19990805,NaN,traditional_hospital,4.131858e+09,1.436118e+08,3.529097e+09,2.198043e+09,56630198.0,228194537.0,113598011.0,22221340.0,2.653634e+08,1.970952e+09,1.963787e+09,0.0,7165400.0,3.553691e+09,2.749986e+09,527179060.0,0.0,2.803885e+09,2.793826e+09,2.590000e+09,1.005862e+07,0.000000e+00,2.720758e+09,0.000000e+00,3.318141e+09,144310353.0,2.982098e+09,1.709276e+09,58244152.0,1.645103e+08,91316429.0,13350570.0,7.211579e+07,1.694659e+09,1.686307e+09,0.0,8351300.0,3.451466e+09,2.778451e+09,410506320.0,0.0,2.658296e+09,2.632717e+09,2.442951e+09,2.557878e+07,0.000000e+00,2.487828e+09,0.000000e+00,85.0,74.0,same
193,281,close,gyeonggi,12,20121213,NaN,NaN,3.054388e+08,2.241614e+07,4.674753e+08,2.548688e+08,13451554.0,90129732.0,4239523.0,0.0,-2.611308e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.342966e+08,64355691.0,8.693647e+08,6.757184e+08,1333.0,2.431707e+07,5929772.0,0.0,-4.237396e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,15.0,15.0,change
206,298,open,gyeongnam,32,20170120,160.0,nursing_hospital,3.279139e+09,3.726966e+08,3.507972e+09,2.659892e+09,1596568.0,83065655.0,83065655.0,0.0,-6.829984e+08,4.237100e+08,3.662513e+08,182604364.0,57458700.0,6.990698e+09,6.972698e+09,8000000.0,0.0,3.478121e+09,6.481208e+08,0.000000e+00,2.830000e+09,2.830000e+09,3.936287e+09,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,103.0,NaN
230,327,open,jeonnam,53,19990309,340.0,general_hospital,2.115853e+08,0.000000e+00,3.794506e+07,0.000000e+00,0.0,0.0,0.0,0.0,1.736403e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00

### 문제 05. 결측치가 포함된 데이터 중 dtype이 object인 column들을 따로 처리하기

In [ ]:
# dtype이 object인 column들을 가져옵니다.
categories = [col for col, dtype in zip(train.columns, train.dtypes) if dtype=='object']
categories

['OC', 'sido', 'instkind', 'ownerChange']

In [ ]:
# 결측치가 있는 데이터 중에서 object column만 뽑아봅시다.
missings[categories]

,OC,sido,instkind,ownerChange
35,close,gyeonggi,nursing_hospital,change
62,open,busan,nursing_hospital,NaN
71,close,jeonbuk,traditional_hospital,same
193,close,gyeonggi,NaN,change
206,open,gyeongnam,nursing_hospital,NaN
230,open,jeonnam,general_hospital,same
297,close,choongnam,hospital,same
299,close,gyeonggi,nursing_hospital,NaN
300,open,seoul,traditional_hospital,same


In [ ]:
# categorical feature의 결측치는 최빈값으로 채웁니다.
modes = train.mode().loc[0]
train[categories] = train[categories].fillna(modes)
train[categories]

,OC,sido,instkind,ownerChange
0,open,choongnam,nursing_hospital,same
2,open,gyeonggi,nursing_hospital,same
3,open,incheon,general_hospital,same
4,open,gyeongnam,general_hospital,same
5,open,gyeongnam,general_hospital,same
...,...,...,...,...
295,open,gyeonggi,nursing_hospital,same
296,open,jeonnam,nursing_hospital,same
297,close,choongnam,hospital,same
299,close,gyeonggi,nursing_hospital,same


In [ ]:
train

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,31641798.0,2.251697e+08,1.012700e+09,9.976719e+08,4.700557e+08,1.502781e+07,2.514586e+09,2.360684e+09,1.434496e+08,0.0,6.828260e+08,2.013237e+08,0.000000e+00,4.815023e+08,3.510000e+08,2.844460e+09,1.496394e+09,4.297848e+09,0.000000e+00,4.057422e+09,2.063787e+09,16194675.0,2.998335e+07,1.568305e+07,18808074.0,2.078297e+08,8.301695e+08,8.165705e+08,5.237026e+08,1.359897e+07,2.548115e+09,2.386263e+09,1.458986e+08,0.0,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,0.0,4.186475e+07,2.724421e+08,2.536822e+08,8.095950e+06,1.875997e+07,1.204810e+08,1.204810e+08,0.000000e+00,0.0,9.241434e+07,9.241434e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.005088e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,173769780.0,9.028303e+08,1.304154e+10,1.153475e+10,0.000000e+00,1.506793e+09,4.317936e+10,3.832078e+10,3.945208e+09,0.0,4.236380e+10,2.799188e+10,1.988520e+10,1.437192e+10,7.253040e+09,1.385710e+10,8.643659e+09,6.685834e+10,0.000000e+00,6.492419e+10,2.971135e+10,476807804.0,1.353672e+09,1.277422e+09,218891720.0,8.383875e+08,1.112572e+10,9.890540e+09,0.000000e+00,1.235181e+09,3.958356e+10,3.485576e+10,3.915906e+09,0.0,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,0.0,8.074970e+07,6.317084e+09,5.873265e+09,4.099320e+09,4.438186e+08,4.366733e+10,4.330613e+10,2.223400e+08,0.0,4.989811e+10,2.890919e+10,1.793038e+10,2.098892e+10,1.350000e+10,8.631164e+07,9.025550e+09,4.808280e+10,0.000000e+00,4.712580e+10,2.346004e+10,597748128.0,1.522108e+09,1.349851e+09,0.0,3.264258e+07,4.906776e+09,4.464017e+09,3.365227e+09,4.427591e+08,4.653138e+10,4.562945e+10,7.893407e+08,0.0,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same
5,11,open,gyeongnam,155,20020501,243.0,general_hospital,3.358054e+10,9.222997e+09,2.372791e+10,1.665533e+10,65414325.0,7.412694e+08,3.364622e+08,0.0,-4.622738e+07,5.635105e+09,5.481680e+09,6.634820e+08,1.534246e+08,1.864970e+10,1.849938e+10,1.364582e+08,0.0,0.000000e+00,1.849938e+10,1.718053e+10,0.000000e+00,0.000000e+00,0.000000e+00,3.664918e+09,3.433445e+10,9.839139e+09,2.409622e+10,1.638792e+10,125681154.0,4.500830e+08,4.155104e+08,0.0,7.468595e+07,4.869419e+09,4.725857e+09,1.976163e+08,1.435618e+08,2.212715e+10,2.186113e+10,1.910460e+08,0.0,0.000000e+00,2.186113e+10,1.708714e+10,2.000000e+07,0.000000e+00,0.000000e+00,5.879004e+09,397.0,390.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### 문제 06. 결측치 채우기

- 모든 숫자 데이터에 대해서 평균값으로 결측치를 채워줍니다.

In [ ]:
train.fillna(train.mean(), inplace=True)

In [ ]:
# 결측치가 제대로 처리되었는지 확인하기
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             286 non-null    int64  
 1   OC                  286 non-null    object 
 2   sido                286 non-null    object 
 3   sgg                 286 non-null    int64  
 4   openDate            286 non-null    int64  
 5   bedCount            286 non-null    float64
 6   instkind            286 non-null    object 
 7   revenue1            286 non-null    float64
 8   salescost1          286 non-null    float64
 9   sga1                286 non-null    float64
 10  salary1             286 non-null    float64
 11  noi1                286 non-null    float64
 12  noe1                286 non-null    float64
 13  interest1           286 non-null    float64
 14  ctax1               286 non-null    float64
 15  profit1             286 non-null    float64
 16  liquidAs

### 문제 07. 테스트 데이터 결측치 확인하기

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             127 non-null    int64  
 1   OC                  0 non-null      float64
 2   sido                127 non-null    object 
 3   sgg                 127 non-null    int64  
 4   openDate            126 non-null    float64
 5   bedCount            119 non-null    float64
 6   instkind            125 non-null    object 
 7   revenue1            125 non-null    float64
 8   salescost1          125 non-null    float64
 9   sga1                125 non-null    float64
 10  salary1             125 non-null    float64
 11  noi1                125 non-null    float64
 12  noe1                125 non-null    float64
 13  interest1           125 non-null    float64
 14  ctax1               125 non-null    float64
 15  profit1             125 non-null    float64
 16  liquidAs

In [ ]:
# OC는 target column이므로 제거
test.drop("OC", axis=1, inplace=True)

### 문제 08. 테스트 데이터에 동일한 결측치 적용하기

In [ ]:
# dtype이 object인 column들을 가져옵니다.
categories = [col for col, dtype in zip(test.columns, test.dtypes) if dtype=='object']
categories.remove('employee1')
categories.remove('employee2')

- 새로운 employee1, employee2가 나와서 이 둘은 나중에 처리하고 처리했었던 나머지 column들을 먼저 처리합니다.

In [ ]:
# test에서 결측치가 있는 데이터들을 뽑아냅니다.
missings2 = test[test.isnull().any(axis=1)]
missings2

,inst_id,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
2,6,jeonnam,103,19890427.0,276.0,general_hospital,2.326031e+10,2.542571e+09,2.308749e+10,1.168734e+10,1.601943e+09,1.380941e+09,8.101204e+08,0.0,-2.148749e+09,1.265313e+10,1.133922e+10,7.337107e+09,1.313914e+09,2.945467e+10,2.830632e+10,1.388070e+08,0.0,0.000000e+00,2.830632e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,2.468035e+10,1.500100e+09,2.373950e+10,1.191146e+10,2.361216e+08,1.328858e+09,6.259234e+08,15210270.0,-1.667198e+09,1.354202e+10,1.263851e+10,7.003703e+09,9.035036e+08,2.938810e+10,2.777589e+10,4.288510e+08,0.0,0.000000e+00,2.777589e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN,NaN
10,24,gyeonggi,169,20071109.0,NaN,traditional_clinic,7.034041e+10,1.570361e+10,5.177418e+10,3.204005e+10,4.465973e+08,1.730254e+09,1.658762e+09,238518390.0,1.340446e+09,9.285804e+09,8.766996e+09,1.308363e+07,5.188082e+08,5.641221e+10,5.513259e+10,9.400293e+08,0.0,5.704074e+10,1.886734e+10,1.096604e+10,3.817340e+10,2.364771e+10,8.657277e+09,7.321695e+09,5.609102e+10,1.275040e+10,4.135528e+10,2.662000e+10,1.604991e+09,2.068379e+09,1.941942e+09,0.0,1.521950e+09,9.677091e+09,9.071467e+09,1.143201e+07,6.056243e+08,6.256096e+10,6.024682e+10,9.805293e+08,0.0,6.241929e+10,1.514333e+10,7.749623e+09,4.727596e+10,3.331840e+10,9.818765e+09,8.483183e+09,532,409,same
13,30,seoul,36,20080801.0,173.0,hospital,6.637116e+09,0.000000e+00,7.465698e+09,3.715042e+09,2.303995e+08,1.903198e+09,1.779581e+09,0.0,-2.501382e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.435991e+09,0.000000e+00,8.509532e+09,3.991395e+09,1.710801e+08,1.652999e+09,1.498246e+09,7460.0,-2.555466e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,same
15,32,busan,64,19950330.0,226.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566,566,same
20,48,daegu,40,19970228.0,154.0,hospital,1.351696e+10,2.469251e+09,1.072392e+10,6.759008e+09,2.733386e+07,1.902601e+08,1.856484e+08,0.0,1.608576e+08,2.127028e+09,2.010007e+09,1.321744e+09,1.170209e+08,1.729681e+10,1.719271e+10,9.105800e+07,0.0,9.176202e+09,9.176202e+09,6.149040e+09,0.000000e+00,0.000000e+00,1.024764e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,150,NaN,NaN
22,64,jeonbuk,34,20060629.0,NaN,hospital,3.207567e+09,2.656878e+07,3.242981e+09,1.628328e+09,3.703870e+05,0.000000e+00,0.000000e+00,0.0,-6.161245e+07,5.021155e+09,4.648823e+09,0.000000e+00,3.723319e+08,3.596652e+10,3.495010e+10,8.771110e+08,0.0,2.436758e+10,9.299760e+09,4.579305e+09,1.506782e+10,9.630291e+09,1.662009e+10,0.000000e+00,3.493287e+09,9.233394e+07,4.614433e+09,1.871574e+09,3.331000e+03,8.426949e+08,8.357097e+08,0.0,-2.056171e+09,5.548918e+09,5.252864e+09,2.651482e+09,2.960539e+08,3.551253e+10,3.437084e+10,9.359910e+08,0.0,2.481421e+10,8.077364e+09,

In [ ]:
# 결측치가 있는 데이터 중에서 object column만 뽑아봅시다.
missings2[categories]

,sido,instkind,ownerChange
2,jeonnam,general_hospital,NaN
10,gyeonggi,traditional_clinic,same
13,seoul,hospital,same
15,busan,general_hospital,same
20,daegu,hospital,NaN
22,jeonbuk,hospital,NaN
40,jeonnam,nursing_hospital,same
52,jeonbuk,nursing_hospital,NaN
56,jeonbuk,nursing_hospital,same
78,gyeongnam,hospital,change


In [ ]:
# 이 데이터들도 최빈값으로 채워줍니다.
test[categories] = test[categories].fillna(modes)
test[categories]

,sido,instkind,ownerChange
0,incheon,general_hospital,same
1,jeju,hospital,same
2,jeonnam,general_hospital,same
3,busan,general_hospital,same
4,jeonbuk,general_hospital,same
...,...,...,...
122,jeonnam,traditional_clinic,same
123,gyeonggi,traditional_hospital,same
124,gyeonggi,hospital,same
125,jeju,nursing_hospital,same


### 문제 09. employee1, employee2 확인하기

In [ ]:
# employee1과 employee2를 봅니다.
test.employee1[:30]

0       693
1       379
2       NaN
3       760
4       437
5       386
6       727
7       377
8        17
9       532
10      532
11      128
12    1,637
13      NaN
14    1,816
15      566
16      104
17      244
18      187
19      431
20      150
21      361
22      NaN
23       41
24      187
25      110
26      457
27      210
28      344
29      250
Name: employee1, dtype: object

In [ ]:
test.employee2[:30]

0       693
1       371
2       NaN
3       760
4       385
5       397
6       580
7       346
8        17
9       409
10      409
11      127
12    1,547
13      NaN
14    1,104
15      566
16      100
17      258
18      165
19      412
20      NaN
21      235
22      NaN
23       41
24      190
25      112
26      457
27      149
28      280
29      260
Name: employee2, dtype: object

In [ ]:
# ','를 없애고 수치값으로 변경합니다.
test.employee1 = test.employee1.astype('str').str.replace(",", "").astype('float')
test.employee2 = test.employee2.astype('str').str.replace(",", "").astype('float')
test

,inst_id,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,2,incheon,139,19981125.0,300.0,general_hospital,6.682486e+10,0.000000e+00,6.565709e+10,3.223695e+10,4.314093e+09,4.901517e+09,1.775872e+09,76680000.0,5.036692e+08,2.591385e+10,2.438741e+10,7.242580e+08,1.526434e+09,4.807282e+10,4.629931e+10,7.463122e+08,0.0,6.769090e+10,5.426088e+10,3.805927e+10,1.343002e+10,6.319097e+09,6.295769e+09,8.944587e+09,6.360375e+10,0.000000e+00,6.261126e+10,3.016946e+10,4.076448e+09,4.332357e+09,1.578626e+09,218399804.0,5.181821e+08,2.498992e+10,2.367928e+10,1.237754e+09,1.310637e+09,4.650191e+10,4.356296e+10,7.963862e+08,0.0,5.540643e+10,5.068443e+10,3.714334e+10,4.720000e+09,4.690000e+09,1.608540e+10,8.944587e+09,693.0,693.0,same
1,5,jeju,149,20160309.0,44.0,hospital,3.495758e+10,0.000000e+00,3.259270e+10,1.672254e+10,6.871024e+07,1.981033e+09,1.936455e+09,0.0,4.525557e+08,5.861727e+09,5.372198e+09,6.610290e+08,4.895291e+08,7.511010e+10,6.678648e+10,4.500000e+06,0.0,6.626737e+10,4.210945e+10,2.330000e+10,2.415791e+10,1.720000e+10,1.470447e+10,1.046277e+10,3.754253e+10,0.000000e+00,3.868523e+10,1.897732e+10,1.918022e+08,3.694365e+09,2.016114e+09,0.0,-4.645265e+09,5.503720e+09,4.943368e+09,1.544307e+08,5.603519e+08,6.559379e+10,6.545302e+10,1.450000e+07,0.0,6.730838e+10,4.209828e+10,2.420000e+10,2.521009e+10,1.830000e+10,3.789135e+09,0.000000e+00,379.0,371.0,same
2,6,jeonnam,103,19890427.0,276.0,general_hospital,2.326031e+10,2.542571e+09,2.308749e+10,1.168734e+10,1.601943e+09,1.380941e+09,8.101204e+08,0.0,-2.148749e+09,1.265313e+10,1.133922e+10,7.337107e+09,1.313914e+09,2.945467e+10,2.830632e+10,1.388070e+08,0.0,0.000000e+00,2.830632e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,2.468035e+10,1.500100e+09,2.373950e+10,1.191146e+10,2.361216e+08,1.328858e+09,6.259234e+08,15210270.0,-1.667198e+09,1.354202e+10,1.263851e+10,7.003703e+09,9.035036e+08,2.938810e+10,2.777589e+10,4.288510e+08,0.0,0.000000e+00,2.777589e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN,same
3,8,busan,71,20100226.0,363.0,general_hospital,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.850652e+09,2.060989e+09,0.000000e+00,0.0,2.789663e+09,8.644533e+09,7.891338e+09,0.000000e+00,7.531944e+08,6.730264e+10,6.415297e+10,0.000000e+00,0.0,2.891880e+10,2.604193e+10,1.875187e+10,2.876879e+09,0.000000e+00,4.702837e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.975530e+09,2.112736e+09,0.000000e+00,0.0,2.862794e+09,1.204897e+10,1.126481e+10,0.000000e+00,7.841551e+08,3.920904e+10,3.601937e+10,0.000000e+00,0.0,1.211517e+10,9.556237e+09,4.251867e+09,2.558931e+09,0.000000e+00,3.914284e+10,0.000000e+00,760.0,760.0,same
4,10,jeonbuk,26,20040604.0,213.0,general_hospital,5.037025e+10,0.000000e+00,4.855803e+10,2.296346e+10,1.534452e+09,0.000000e+00,2.573804e+09,0.0,3.346675e+09,7.849772e+09,7.138327e+09,1.104000e+08,7.114450e+08,6.404854e+10,6.024301e+10,3.582019e+09,0.0,4.921637e+10,2.503353e+10,1.720219e+10,2.418283e+10,9.976421e+09,2.268194e+10,1.492691e+10,4.348996e+10,0.000000e+00,4.067810e+10,1.871616e+10,1.802199e+09,0.000000e+00,3.925279e+09,0.0,4.614056e+09,8.847725e+09,7.868784e+09,1.764000e+08,9.789409e+08,6.085689e+10,6.014111e+10,5.467489e+08,0.0,4.395973e+10,7.535567e+09,3.298427e+09,3.642417e+10,2.134712e+10,2.574488e+10,1.507269e+10,437.0,385.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### 문제 10. 결측치 채우기

In [ ]:
# train 데이터와 동일하게 수치값들은 평균으로 채워줍니다.
test.fillna(train.mean(), inplace=True)

In [ ]:
# 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 57 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             127 non-null    int64  
 1   sido                127 non-null    object 
 2   sgg                 127 non-null    int64  
 3   openDate            127 non-null    float64
 4   bedCount            127 non-null    float64
 5   instkind            127 non-null    object 
 6   revenue1            127 non-null    float64
 7   salescost1          127 non-null    float64
 8   sga1                127 non-null    float64
 9   salary1             127 non-null    float64
 10  noi1                127 non-null    float64
 11  noe1                127 non-null    float64
 12  interest1           127 non-null    float64
 13  ctax1               127 non-null    float64
 14  profit1             127 non-null    float64
 15  liquidAsset1        127 non-null    float64
 16  quickAss

## Step 3. Feature Engineering

- 분석에 사용할 feature들을 만들고, 학습을 위한 데이터를 준비하는 단계입니다.

- 모든 categorical feature들을 변환하여 데이터를 feature vector로 만들어봅니다.

### 문제 11. categorical feature들 확인하기

In [ ]:
# train, test categorical feature들 확인하기
train_categories = [col for col, dtype in zip(train.columns, train.dtypes) if dtype=='object']
test_categories = [col for col, dtype in zip(test.columns, test.dtypes) if dtype=='object']

In [ ]:
print(train_categories)
print(test_categories)

['OC', 'sido', 'instkind', 'ownerChange']
['sido', 'instkind', 'ownerChange']


### 문제 12. training data에 categorical feature encoding

- OneHotEncoder를 이용하여 주어진 feature들을 변환합니다.


- OneHotEncoding은 pd.get_dummies 함수를 사용합니다.


- "OC"는 target column이므로, LabelEncoder로 변환합니다.

In [ ]:
# 라이브러리 불러오기
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

le = LabelEncoder()
ohe = OneHotEncoder()

In [ ]:
# target column 제거
train_categories.remove("OC")

In [ ]:
# One-Hot Encoder로 변환하기
encoded = ohe.fit_transform(X=train[train_categories])
encoded.toarray()

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [ ]:
# Label Encoder로 변환하기
labels = le.fit_transform(train.OC)
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [ ]:
# target column 제거
train.drop("OC", axis=1, inplace=True)

### 문제 13. test data에 동일하게 적용하기

In [ ]:
test_encoded = ohe.transform(X=test[test_categories])

ValueError: ignored

- sido column에 "jeju"라는 값을 가진 새로운 데이터가 등장했습니다.


- 이 데이터는 학습 데이터에는 존재하지 않았던, 종류라서 trainer가 처리를 해줄 수 가 없습니다.


- 이런 경우는 설제 서비스시 예외처리를 해주는 것이 맞습니다.


- 따라서, 저희도 제외를 하고 가는 것이 맞습니다.

In [ ]:
# jeju 확인
jejus = test[test.sido == "jeju"]
jejus

,inst_id,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
1,5,jeju,149,20160309.0,44.000000,hospital,3.495758e+10,0.000000e+00,3.259270e+10,1.672254e+10,68710235.0,1.981033e+09,1.936455e+09,0.0,4.525557e+08,5.861727e+09,5.372198e+09,661028976.0,4.895291e+08,7.511010e+10,6.678648e+10,4.500000e+06,0.0,6.626737e+10,4.210945e+10,2.330000e+10,2.415791e+10,1.720000e+10,1.470447e+10,1.046277e+10,3.754253e+10,0.000000e+00,3.868523e+10,1.897732e+10,191802228.0,3.694365e+09,2.016114e+09,0.0,-4.645265e+09,5.503720e+09,4.943368e+09,154430728.0,5.603519e+08,6.559379e+10,6.545302e+10,1.450000e+07,0.0,6.730838e+10,4.209828e+10,2.420000e+10,2.521009e+10,1.830000e+10,3.789135e+09,0.0,379.0,371.0,same
115,398,jeju,149,20070912.0,13.000000,nursing_hospital,7.492834e+09,0.000000e+00,6.547070e+09,2.527670e+09,148120856.0,6.018288e+08,1.935317e+08,141779000.0,3.502774e+08,2.292805e+09,2.219605e+09,765749755.0,7.320049e+07,8.309067e+09,7.513837e+09,3.112500e+07,0.0,6.938766e+09,2.714969e+08,0.000000e+00,6.667269e+09,5.646644e+09,3.663106e+09,0.000000e+00,7.280190e+09,0.000000e+00,5.952266e+09,2.540483e+09,196931214.0,7.892458e+08,1.165815e+08,64761260.0,6.708481e+08,1.779102e+09,1.717013e+09,998679520.0,6.208833e+07,8.372412e+09,7.452795e+09,3.045000e+07,0.0,6.934269e+09,4.767853e+08,0.000000e+00,6.457484e+09,5.786636e+09,3.217245e+09,0.0,66.0,68.0,same
125,430,jeju,76,20010201.0,145.106383,nursing_hospital,4.892710e+10,4.157148e+10,4.721485e+09,1.514547e+09,85098148.0,1.270081e+09,8.379266e+08,31802061.0,1.417343e+09,1.028286e+10,9.002630e+09,15569326.0,1.280235e+09,2.952030e+10,8.782117e+09,7.537349e+09,0.0,3.859089e+10,9.342814e+09,0.000000e+00,2.924807e+10,2.563120e+10,1.212281e+09,0.000000e+00,4.758477e+10,4.015853e+10,5.061219e+09,1.404341e+09,99795507.0,1.274426e+09,8.433387e+08,0.0,1.190396e+09,7.848233e+09,6.707102e+09,15569326.0,1.141131e+09,2.915067e+10,7.869001e+09,7.112780e+09,0.0,3.720397e+10,8.485918e+09,0.000000e+00,2.871805e+10,2.563120e+10,-2.050629e+08,0.0,363.0,343.0,same


In [ ]:
# jeju 지우기 ### 실제로는 여기서 Report를 하여 학습데이터에 jeju를 추가하는 방향으로 해야한다.
test.drop(jejus.index, inplace=True)

In [ ]:
# 다시 One-Hot encoding 적용
test_encoded = ohe.transform(X=test[test_categories])
test_encoded

<124x25 sparse matrix of type '<class 'numpy.float64'>'
	with 372 stored elements in Compressed Sparse Row format>

### 문제 14. feature vector와 target vector 만들기

In [ ]:
# shape 확인하기
print(train.shape, encoded.toarray().shape)

(286, 57) (286, 25)


In [ ]:
# 변환될 column을 지우고, OHE된 벡터들을 합친다.
train.drop(train_categories, axis=1, inplace=True)
X_train = np.hstack([train.values, encoded.toarray()])
y_train = labels

print(X_train.shape, y_train.shape)

(286, 79) (286,)


In [ ]:
# test 데이터도 확인하기
print(test.shape)

(124, 57)


In [ ]:
# test 데이터도 동일한 방법으로 진행
test.drop(test_categories, axis=1, inplace=True)
X_test = np.hstack([test.values, test_encoded.toarray()])
X_test.shape

(124, 79)

### 문제 15. feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 스케일 조정된 X_train 데이터 확인하기
X_train_scaled 

array([[-1.82765709, -0.14428866,  0.21838383, ..., -0.15839699,
        -0.41488691,  0.41488691],
       [-1.8024622 ,  0.16818882,  1.24922335, ..., -0.15839699,
        -0.41488691,  0.41488691],
       [-1.77726731,  1.18374062, -0.5881221 , ..., -0.15839699,
        -0.41488691,  0.41488691],
       ...,
       [ 1.71642371,  1.53527778,  1.0131611 , ..., -0.15839699,
        -0.41488691,  0.41488691],
       [ 1.7500169 ,  1.73057621,  0.20547543, ..., -0.15839699,
        -0.41488691,  0.41488691],
       [ 1.75841519, -0.02710961, -0.47125926, ...,  6.31325137,
        -0.41488691,  0.41488691]])

### 참고: scikit-learn에서 제공하는 피처 스케일러(scaler)

- `StandardScaler`: 기본 스케일, 각 피처의 평균을 0, 표준편차를 1로 변환
- `RobustScaler`: 위와 유사하지만 평균 대신 중간값(median)과 일분위, 삼분위값(quartile)을 사용하여 이상치 영향을 최소화
- `MinMaxScaler`: 모든 피처의 최대치와 최소치가 각각 1, 0이 되도록 스케일 조정
- `Normalizer`: 피처(컬럼)이 아니라 row마다 정규화되며, 유클리드 거리가 1이 되도록 데이터를 조정하여 빠르게 학습할 수 있게 함

<p> 스케일 조정을 하는 이유는 데이터의 값이 너무 크거나 작을 때 학습이 제대로 되지 않을 수도 있기 때문입니다. 또한 스케일의 영향이 절대적인 분류기(예: knn과 같은 거리기반 알고리즘)의 경우, 스케일 조정을 필수적으로 검토해야 합니다.
    
<p> 반면 어떤 항목은 원본 데이터의 분포를 유지하는 것이 나을 수도 있습니다. 예를 들어, 데이터가 거의 한 곳에 집중되어 있는 feature를 표준화시켜 분포를 같게 만들었을 때, 작은 단위의 변화가 큰 차이를 나타내는 것처럼 학습될 수도 있습니다. 또한 스케일의 영향을 크게 받지 않는 분류기(예: 트리 기반 앙상블 알고리즘)를 사용할 경우에도 성능이 준수하게 나오거나 과대적합(overfitting)의 우려가 적다면 생략할 수도 있습니다.
    
<p> 스케일 조정시 유의해야할 점은 원본 데이터의 의미를 잃어버릴 수 있다는 것입니다. 최종적으로 답을 구하는 것이 목적이 아니라 모델의 해석이나 향후 다른 데이터셋으로의 응용이 더 중요할 때 원 피처에 대한 설명력을 잃어버린다면 모델 개선이 어려울 수도 있습니다. 이 점을 함께 고려하시면 좋겠습니다.

## Step 4. Training

- 공부한 각 머신러닝 분류 모델들을 돌려보고, 성능을 평가합니다.

- 아래 모델들이 그 대상에 해당합니다.

- Logistic Regression / Naive Bayes / Decision Tree / Random Forest / KNN / SVM / xgboost / lightgbm

### 문제 16. Logistic Regression 모델 돌려보기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 문제 17. Logistic Regression 모델의 정확도 평가하기

In [ ]:
pred = log_reg.predict(X_train_scaled)
logreg_score = accuracy_score(y_train, pred)

print('Logistic Regression accuracy score: {0:0.4f}'.format(logreg_score))

Logistic Regression accuracy score: 0.9755


### 문제 18. Naive Bayes 모델 돌려보기

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_scaled, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

### 문제 19. Naive Bayes 평가하기

In [ ]:
pred = nb.predict(X_train_scaled)
nb_score = accuracy_score(y_train, pred)

print('Naive Bayes accuracy score: {0:0.4f}'.format(nb_score))

Naive Bayes accuracy score: 0.7378


### 문제 20. Decision Tree 모델 돌려보기

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=8)
dt.fit(X_train_scaled, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

### 문제 21. Decision Tree 모델 평가하기

In [ ]:
pred = dt.predict(X_train_scaled)
dt_score = accuracy_score(y_train, pred)

print('Decision Tree accuracy score: {0:0.4f}'.format(dt_score))

Decision Tree accuracy score: 0.9965


### 문제 22. Random Forest 모델 돌려보기

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=15, max_features=0.8)  
rf.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=0.8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred = rf.predict(X_train_scaled) 
rf_score = accuracy_score(y_train, pred)
print('Random Forest Model accuracy score : {0:0.4f}'.format(rf_score))

Random Forest Model accuracy score : 1.0000


### 문제 23. KNN Classification 모델 돌려보기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
pred = knn.predict(X_train_scaled) 
knn_score = accuracy_score(y_train, pred)
print('KNN classification accuracy score : {0:0.4f}'.format(knn_score))

KNN classification accuracy score : 0.9615


### 문제 24. Support Vector Machine 모델 돌려보기

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train_scaled, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred = svc.predict(X_train_scaled) 
svm_score = accuracy_score(y_train, pred)
print('Support Vector Machine accuracy score : {0:0.4f}'.format(svm_score))

Support Vector Machine accuracy score : 0.9615


### 문제 25. XGBoost 모델 돌려보기

- xgboost의 구현체가 많지만, 여기서는 sklearn version을 사용합니다.

In [ ]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### 문제 26. XGBoost 모델의 분류결과 확인하기

In [ ]:
pred = xgb.predict(X_train_scaled) 
xgb_score = accuracy_score(y_train, pred)
print('XGBoost accuracy score : {0:0.4f}'.format(xgb_score))

XGBoost accuracy score : 0.9930


### 문제 27. Light GBM 모델 돌려보기

In [ ]:
from lightgbm.sklearn import LGBMClassifier

lgb = LGBMClassifier()
lgb.fit(X_train_scaled, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### 문제 28. Light GBM 분류결과 확인하기

In [ ]:
pred = lgb.predict(X_train_scaled) 
lgb_score = accuracy_score(y_train, pred)
print('XGBoost accuracy score : {0:0.4f}'.format(lgb_score))

XGBoost accuracy score : 0.9965


### 참고 : 부스팅(Boosting) 모델 개요


- 부스팅은 여러 트리의 적합 결과를 합하는 앙상블 알고리즘의 하나로, 이 때 sequential의 개념이 추가되어 있습니다. 즉 연속적인 weak learner, 바로 직전 weak learner의 error를 반영한 현재 weak learner를 잡겠다는 것입니다. 이 아이디어는 Gradient Boosting Model(GBM)에서 loss를 계속 줄이는 방향으로 weak learner를 잡는다는 개념으로 확장됩니다.

![boost](https://pluralsight2.imgix.net/guides/81232a78-2e99-4ccc-ba8e-8cd873625fdf_2.jpg)


- 부스팅 계열 모델은 XGBoost, LightGBM, CatBoost 등이 있습니다.


- 더 자세한 내용은 다음 Step에서 살펴보겠습니다.

### 문제 29. Test prediction

- 학습한 모델을 기반하여 test 데이터를 평가합니다. 그 예측값이 대회 제출이 되는 기준입니다.


- 실제 현업에서는 서비스에 들어가는 기술을 test 데이터로 평가합니다. 마치 유저가 들어온 것처럼요.

In [ ]:
# 모델을 하나 정해서, test data에 대한 inference를 진행합니다.
final = lgb.predict(X_test_scaled)
final

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# 결과 파일 생성
result = pd.DataFrame({"ID":test.index, "target":final}).set_index("ID")
result

,target
ID,
0,1
2,1
3,1
4,1
5,1
...,...
121,1
122,1
123,1


### 문제 30. 최종 성능 정리

In [ ]:
print('Accuracy Comparisons for Binary Models\n')
print('logreg_score:', '{0:0.5f}'.format(logreg_score))
print('dt_score   :', '{0:0.5f}'.format(dt_score))
print('rf_score   :', '{0:0.5f}'.format(rf_score))
print('knn_score   :', '{0:0.5f}'.format(knn_score))
print('svm_score   :', '{0:0.5f}'.format(svm_score))
print('xgb_score   :', '{0:0.5f}'.format(xgb_score))
print('lgb_score   :', '{0:0.5f}'.format(lgb_score))

Accuracy Comparisons for Binary Models

logreg_score: 0.97552
dt_score   : 0.99650
rf_score   : 1.00000
knn_score   : 0.96154
svm_score   : 0.96154
xgb_score   : 0.99301
lgb_score   : 0.99650
